# Chapter 2: Emitting Basic MLIR

This is an xDSL version of the Toy compiler, as described in the [MLIR tutorial](https://mlir.llvm.org/docs/Tutorials/Toy/)

This tutorial will be illustrated with a toy language that we’ll call “Toy” (naming is hard…). Toy is a tensor-based language that allows you to define functions, perform some math computation, and print results.

In [1]:
from xdsl.ir import MLContext, SSAValue, Operation
from xdsl.dialects.toy import *
from xdsl.dialects.builtin import ModuleOp
from typing import List


In [2]:
%load_ext autoreload
%autoreload 2

from xdsl.printer import Printer

In [3]:
# MLContext, containing information about the registered dialects
context = MLContext()

toy = Toy(context)

# Printer used to pretty-print MLIR data structures
printer = Printer(target=Printer.Target.MLIR)

In [4]:
def func_body(arg0: SSAValue, arg1: SSAValue) -> List[Operation]:
    f0 = TransposeOp.from_input(arg0)
    f1 = TransposeOp.from_input(arg1)
    f2 = MulOp.from_summands(f0.results[0], f1.results[0])
    f3 = ReturnOp.from_input(f2.results[0])
    return [f0, f1, f2, f3]

def main_body() -> List[Operation]:
    m0 = ConstantOp.from_list([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], [2, 3])
    m1 = ReshapeOp.from_input(m0.results[0], [2,3])
    m2 = ConstantOp.from_list([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], [2, 3])
    m3 = ReshapeOp.from_input(m2.results[0], [2,3])
    m4 = GenericCallOp.get('multiply_transpose', [m1.results[0], m3.results[0]], [TensorType.from_type_and_list(f64, [2, 3])])
    m5 = GenericCallOp.get('multiply_transpose', [m3.results[0], m1.results[0]], [TensorType.from_type_and_list(f64, [2, 3])])
    m6 = PrintOp.from_input(m5.results[0])
    m7 = ReturnOp.from_input(m5.results[0])
    return [
        m0, m1, m2, m3, m4, m5, m6, m7
    ]

multiply_transpose = FuncOp.from_callable('multiply_transpose', [TensorTypeF64, TensorTypeF64], [TensorTypeF64], func_body)
main = FuncOp.from_callable('main', [], [], main_body)

module = ModuleOp.from_region_or_ops([multiply_transpose, main])

printer.print(module)

"builtin.module"() ({
  "toy.func"() ({
  ^0(%0 : tensor<1xf64>, %1 : tensor<1xf64>):
    %2 = "toy.transpose"(%0) : (tensor<1xf64>) -> tensor<1xf64>
    %3 = "toy.transpose"(%1) : (tensor<1xf64>) -> tensor<1xf64>
    %4 = "toy.mul"(%2, %3) : (tensor<1xf64>, tensor<1xf64>) -> tensor<1xf64>
    "toy.return"(%4) : (tensor<1xf64>) -> ()
  }) {sym_name = "multiply_transpose", function_type = (tensor<1xf64>, tensor<1xf64>) -> tensor<1xf64>} : () -> ()
  "toy.func"() ({
    %5 = "toy.constant"() {value = dense<[[1.000000e+00, 2.000000e+00, 3.000000e+00], [4.000000e+00, 5.000000e+00, 6.000000e+00]]> : tensor<2x3xf64>} : () -> tensor<2x3xf64>
    %6 = "toy.reshape"(%5) : (tensor<2x3xf64>) -> tensor<2x3xf64>
    %7 = "toy.constant"() {value = dense<[[1.000000e+00, 2.000000e+00, 3.000000e+00], [4.000000e+00, 5.000000e+00, 6.000000e+00]]> : tensor<2x3xf64>} : () -> tensor<2x3xf64>
    %8 = "toy.reshape"(%7) : (tensor<2x3xf64>) -> tensor<2x3xf64>
    %9 = "toy.generic_call"(%6, %8) {callee = @mult

In [5]:


const_op = ConstantOp.from_list([1.0, 2.0, 3.0, 4.0, 5.0, 6.0], [2, 3])
printer.print_op(const_op)

%11 = "toy.constant"() {value = dense<[[1.000000e+00, 2.000000e+00, 3.000000e+00], [4.000000e+00, 5.000000e+00, 6.000000e+00]]> : tensor<2x3xf64>} : () -> tensor<2x3xf64>


In [6]:
# %0 : !tensor<[2 : !index, 3 : !index], !f64> = toy.constant() ["value" = !dense<!tensor<[2 : !index, 3 : !index], !f64>, [1.0 : !f64, 2.0 : !f64, 3.0 : !f64, 4.0 : !f64, 5.0 : !f64, 6.0 : !f64]>]
# %0 = "toy.constant"() {"value" = dense<[1.0, 2.0, 3.0, 4.0, 5.0, 6.0]> : tensor<2x3x!f64>} : () -> tensor<2x3x!f64>


In [7]:
from xdsl.dialects.builtin import IntAttr

my_int = IntAttr.from_int(42)
printer.print_attribute(my_int)

#int<42>

In [8]:
from pathlib import Path

ast_toy = Path() / 'toy' / 'ast.toy'

In [9]:
with open(ast_toy, 'r') as f:
    print(f.read())

# RUN: toyc-ch1 %s -emit=ast 2>&1 | FileCheck %s

# User defined generic function that operates on unknown shaped arguments.
def multiply_transpose(a, b) {
  return transpose(a) * transpose(b);
}

def main() {
  # Define a variable `a` with shape <2, 3>, initialized with the literal value.
  # The shape is inferred from the supplied literal.
  var a = [[1, 2, 3], [4, 5, 6]];
  # b is identical to a, the literal array is implicitly reshaped: defining new
  # variables is the way to reshape arrays (element count in literal must match
  # the size of specified shape).
  var b<2, 3> = [1, 2, 3, 4, 5, 6];

  # This call will specialize `multiply_transpose` with <2, 3> for both
  # arguments and deduce a return type of <2, 2> in initialization of `c`.
  var c = multiply_transpose(a, b);
  # A second call to `multiply_transpose` with <2, 3> for both arguments will
  # reuse the previously specialized and inferred version and return `<2, 2>`
  var d = multiply_transpose(b, a);
  # A new call w

In [10]:
from toy.Parser import Parser

with open(ast_toy, 'r') as f:
    parser = Parser(ast_toy, f.read())

print(parser.parseModule().dump())

Module:
  Function 
    Proto 'multiply_transpose' toy/ast.toy:4:1'
    Params: [a, b]
    Block {
      Return
        BinOp: * toy/ast.toy:5:25
          Call 'transpose' [ toy/ast.toy:5:10
            var: a toy/ast.toy:5:20
          ]
          Call 'transpose' [ toy/ast.toy:5:25
            var: b toy/ast.toy:5:35
          ]
    } // Block
  Function 
    Proto 'main' toy/ast.toy:8:1'
    Params: []
    Block {
      VarDecl a<> toy/ast.toy:11:3
        Literal: <2, 3>[ <3>[ 1.000000e+00, 2.000000e+00, 3.000000e+00], <3>[ 4.000000e+00, 5.000000e+00, 6.000000e+00]] toy/ast.toy:11:11
      VarDecl b<2, 3> toy/ast.toy:15:3
        Literal: <6>[ 1.000000e+00, 2.000000e+00, 3.000000e+00, 4.000000e+00, 5.000000e+00, 6.000000e+00] toy/ast.toy:15:17
      VarDecl c<> toy/ast.toy:19:3
        Call 'multiply_transpose' [ toy/ast.toy:19:11
          var: a toy/ast.toy:19:30
          var: b toy/ast.toy:19:33
        ]
      VarDecl d<> toy/ast.toy:22:3
        Call 'multiply_transpose' [ t

In [11]:
def hello(bla):
    print(bla)

bla = lambda x: print(x)

bla.__name__

'<lambda>'